In [1]:
import pandas as pd
import numpy as np
import json
import re

In [2]:
with open("2018-02-14_focus/2018-02-14_focus_group.json") as focus_group:
    focus = json.load(focus_group)

In [3]:
with open("krx_code.json") as krx:
    krx_code = json.load(krx)

In [4]:
with open("2018-02-14_opening_increase.json") as oi:
    increase = json.load(oi)

In [5]:
import bs4
import urllib3
import re

def opening_increase(code, date):
    '''
    code: string
    date: string, e.g. 2018-02-14
    '''

    target = "http://finance.naver.com/item/sise_time.nhn?code=" + code + "&thistime=" + re.sub('[-]', '', date) + "090001&page=1"
    pm = urllib3.PoolManager()
    html = pm.urlopen(url=target, method="GET").data
    soup = bs4.BeautifulSoup(html, 'lxml')
    
    data_list = soup.find_all("tr")[2].find_all("td",class_="num")
    price = data_list[0].text
    
    return price != '\xa0' and "상승" in str(soup)

In [6]:
'''
rv = []
c = 0
for stock in focus:
    c+= 1
    if opening_increase(krx_code[stock], "2018-02-14"):
        rv.append(stock)
    print(c)
'''

'\nrv = []\nc = 0\nfor stock in focus:\n    c+= 1\n    if opening_increase(krx_code[stock], "2018-02-14"):\n        rv.append(stock)\n    print(c)\n'

In [7]:
'''
json = json.dumps(rv, ensure_ascii=False)
f = open("2018-02-14_opening_increase.json","w")
f.write(json)
f.close()
'''

'\njson = json.dumps(rv, ensure_ascii=False)\nf = open("2018-02-14_opening_increase.json","w")\nf.write(json)\nf.close()\n'

In [8]:
'''
def ten_minutes_before(time):
    
    if time[-2:] == "00":
        prefix = '0' + str(int(time[11:13]) - 1) + ":50" if int(time[11:13]) <= 10 \
        else str(int(time[11:13]) - 1) + ":50"
        time = time[:-5] + prefix
    
    else:
        time = time[:-2] + str(int(time[-2:]) - 10)
    
    return time
'''

'\ndef ten_minutes_before(time):\n    \n    if time[-2:] == "00":\n        prefix = \'0\' + str(int(time[11:13]) - 1) + ":50" if int(time[11:13]) <= 10         else str(int(time[11:13]) - 1) + ":50"\n        time = time[:-5] + prefix\n    \n    else:\n        time = time[:-2] + str(int(time[-2:]) - 10)\n    \n    return time\n'

In [9]:
import bs4
import urllib3

def scrape_price_history(code, time):
    '''
    code: string
    time: string, e.g. 201802090910
    '''

    target = "http://finance.naver.com/item/sise_time.nhn?code=" + code + "&thistime=" + time + "01&page=1"
    pm = urllib3.PoolManager()
    html = pm.urlopen(url=target, method="GET").data
    soup = bs4.BeautifulSoup(html, 'lxml')
    data_list = soup.find_all("tr")[2].find_all("td",class_="num")
    data_dic = {}
    data_dic["price"] = data_list[0].text
    image = data_list[1].find("img")
    if image != None:
        if image["alt"] == "상승":
            data_dic["price_dif"] = data_list[1].text.strip("\n\t")
        else:
            data_dic["price_dif"] = "-" + data_list[1].text.strip("\n\t")
    else:
        data_dic["price_dif"] = 0
    data_dic["sell"] = data_list[2].text
    data_dic["buy"] = data_list[3].text
    data_dic["volume"] = data_list[4].text
    data_dic["variation"] = data_list[5].text
    
    return data_dic

In [10]:
time = []

prefix = "2018-02-14_focus/discussion_2018-02-14"
for hour in range(9,10):
    for minute in range(0, 3):
        time.append(prefix + " " + (("0" + str(hour)) if hour <= 9 else str(hour))+ ":" + str(minute) + "0.json")

In [11]:
time

['2018-02-14_focus/discussion_2018-02-14 09:00.json',
 '2018-02-14_focus/discussion_2018-02-14 09:10.json',
 '2018-02-14_focus/discussion_2018-02-14 09:20.json']

In [12]:
df = pd.DataFrame(columns=[["code", "name", "post_num_0", "unique_id_0", "click_0", "like_0",\
                            "dislike_0", "time_0","price_0","price_dif_0",\
                            "sell_0", "buy_0", "volume_0", "variation_0",
                            "post_num_1", "unique_id_1", "click_1", "like_1",\
                            "dislike_1", "time_1","price_1","price_dif_1",\
                            "sell_1", "buy_1", "volume_1", "variation_1",
                            "post_num_2", "unique_id_2", "click_2", "like_2",\
                            "dislike_2", "time_2","price_2","price_dif_2",\
                            "sell_2", "buy_2", "volume_2", "variation_2"]])

In [13]:
for t in time:
    with open(t) as f:
        discussion = json.load(f)
    for d in discussion:
        if d["name"] in increase:
            row = pd.DataFrame(columns=["code", "name", "post_num_0", "unique_id_0", "click_0", "like_0",\
                                "dislike_0", "time_0","price_0","price_dif_0",\
                                "sell_0", "buy_0", "volume_0", "variation_0",
                                "post_num_1", "unique_id_1", "click_1", "like_1",\
                                "dislike_1", "time_1","price_1","price_dif_1",\
                                "sell_1", "buy_1", "volume_1", "variation_1",
                                "post_num_2", "unique_id_2", "click_2", "like_2",\
                                "dislike_2", "time_2","price_2","price_dif_2",\
                                "sell_2", "buy_2", "volume_2", "variation_2"],\
                           data = [[krx_code[d["name"]], d["name"], d["post_num"], d["unique_id"], d["click"], \
                                    d["like"], d["dislike"], d["time"], \
                                    np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,\
                                    np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,\
                                    np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]])
            df = df.append(row)

In [14]:
df = df.reset_index()

In [15]:
df.shape

(183, 39)

In [16]:
for idx, row in df.iterrows():
    timestamp = row["time_0"]
    t = re.sub('[ :-]', '', timestamp)
    
    d = scrape_price_history(row["code"], t)
    tpl = (d["price"], d["price_dif"], d["sell"], d["buy"], d["volume"], d["variation"])
    
    df["price_0"].iloc[idx] = tpl[0]
    df["price_dif_0"].iloc[idx] = tpl[1]
    df["sell_0"].iloc[idx] = tpl[2]
    df["buy_0"].iloc[idx] = tpl[3]
    df["volume_0"].iloc[idx] = tpl[4]
    df["variation_0"].iloc[idx] = tpl[5]
    
    print(idx)

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182


In [17]:
for idx, row in df.iterrows():
    if idx >= len(increase) * 1:
        df["post_num_1"].iloc[idx] = df["post_num_0"].iloc[idx - len(increase)]
        df["unique_id_1"].iloc[idx] = df["unique_id_0"].iloc[idx - len(increase)]
        df["click_1"].iloc[idx] = df["click_0"].iloc[idx - len(increase)]
        df["like_1"].iloc[idx] = df["like_0"].iloc[idx - len(increase)]
        df["dislike_1"].iloc[idx] = df["dislike_0"].iloc[idx - len(increase)]
        df["time_1"].iloc[idx] = df["time_0"].iloc[idx - len(increase)]
        
        df["price_1"].iloc[idx] = df["post_num_0"].iloc[idx - len(increase)]
        df["price_dif_1"].iloc[idx] = df["price_dif_0"].iloc[idx - len(increase)]
        df["sell_1"].iloc[idx] = df["sell_0"].iloc[idx - len(increase)]
        df["buy_1"].iloc[idx] = df["buy_0"].iloc[idx - len(increase)]
        df["volume_1"].iloc[idx] = df["volume_0"].iloc[idx - len(increase)]
        df["variation_1"].iloc[idx] = df["variation_0"].iloc[idx - len(increase)]

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [18]:
for idx, row in df.iterrows():
    if idx >= len(increase) * 2:
        df["post_num_2"].iloc[idx] = df["post_num_0"].iloc[idx - (len(increase) * 2)]
        df["unique_id_2"].iloc[idx] = df["unique_id_0"].iloc[idx - (len(increase) * 2)]
        df["click_2"].iloc[idx] = df["click_0"].iloc[idx - (len(increase) * 2)]
        df["like_2"].iloc[idx] = df["like_0"].iloc[idx - (len(increase) * 2)]
        df["dislike_2"].iloc[idx] = df["dislike_0"].iloc[idx - (len(increase) * 2)]
        df["time_2"].iloc[idx] = df["time_0"].iloc[idx - (len(increase) * 2)]
        
        df["price_2"].iloc[idx] = df["post_num_0"].iloc[idx - (len(increase) * 2)]
        df["price_dif_2"].iloc[idx] = df["price_dif_0"].iloc[idx - (len(increase) * 2)]
        df["sell_2"].iloc[idx] = df["sell_0"].iloc[idx - (len(increase) * 2)]
        df["buy_2"].iloc[idx] = df["buy_0"].iloc[idx - (len(increase) * 2)]
        df["volume_2"].iloc[idx] = df["volume_0"].iloc[idx - (len(increase) * 2)]
        df["variation_2"].iloc[idx] = df["variation_0"].iloc[idx - (len(increase) * 2)]

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [19]:
df

,index,code,name,post_num_0,unique_id_0,click_0,like_0,dislike_0,time_0,price_0,...,click_2,like_2,dislike_2,time_2,price_2,price_dif_2,sell_2,buy_2,volume_2,variation_2
0,0,008770,호텔신라,3,2,77,4,2,2018-02-14 09:00,"84,300",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,010170,대한광통신,0,0,0,0,0,2018-02-14 09:00,"7,040",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,022100,포스코 ICT,2,2,451,1,3,2018-02-14 09:00,"8,350",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,009830,한화케미칼,1,1,317,2,7,2018-02-14 09:00,"32,550",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,204840,지엘팜텍,3,3,220,4,6,2018-02-14 09:00,"4,790",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0,033290,코웰패션,1,1,48,0,0,2018-02-14 09:00,"5,270",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0,003490,대한항공,0,0,0,0,0,2018-02-14 09:00,"35,700",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0,054670,대한뉴팜,0,0,0,0,0,2018-02-14 09:00,"13,850",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0,038390,레드캡투어,0,0,0,0,0,2018-02-14 09:00,"15,350",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0,139480,이마트,0,0,0,0,0,2018-02-14 09:00,"284,500",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
df.columns

Index(['index', 'code', 'name', 'post_num_0', 'unique_id_0', 'click_0',
       'like_0', 'dislike_0', 'time_0', 'price_0', 'price_dif_0', 'sell_0',
       'buy_0', 'volume_0', 'variation_0', 'post_num_1', 'unique_id_1',
       'click_1', 'like_1', 'dislike_1', 'time_1', 'price_1', 'price_dif_1',
       'sell_1', 'buy_1', 'volume_1', 'variation_1', 'post_num_2',
       'unique_id_2', 'click_2', 'like_2', 'dislike_2', 'time_2', 'price_2',
       'price_dif_2', 'sell_2', 'buy_2', 'volume_2', 'variation_2'],
      dtype='object')

In [56]:
rv = []
rv.append(df.columns.tolist())
for row in df.iterrows():
    rv.append(row[1].tolist())

In [57]:
'''
import json
json = json.dumps(rv, ensure_ascii=False)
f = open("2018-02-14_09:00_09:20_dataframe.json","w")
f.write(json)
f.close()
'''